In [1]:
import pandas as pd
import time
import datetime
import scipy.stats as st
import numpy as np
import os
import ipyparallel as ipp

# 准备数据

In [2]:
# prepare data
#msgat = pd.read_pickle(r'../records/sample/msgat_dataframe')
total_msg = pd.read_pickle(r'../records/sample/sample_msg_tfidf_pickle')
# drop useless columns
total_msg.drop(columns=['type', 'status', 'isSend', 'splitwords','msg'], inplace=True)
# test a subsample of total_msg
total_msg.sort_values('createTime',inplace= True)
total_msg['intTime'] = total_msg.createTime.diff().dt.total_seconds().fillna(0).cumsum()
total_msg.sort_values(['talker','createTime'],inplace= True,ascending=True)
total_msg.index = pd.RangeIndex(len(total_msg.index))
total_msg['talker'] = total_msg['talker'].astype('category')
total_msg['sender'] = total_msg['sender'].astype('category')
total_msg['displayname'] = total_msg['displayname'].astype('category')
total_msg['nickname'] = total_msg['nickname'].astype('category')
total_msg['createDate'] = total_msg['createTime'].dt.date
total_msg['createDate'] = total_msg['createDate'].astype('category')
#total_msg = total_msg.iloc[:10000]
msgat = pd.read_pickle(r'../records/sample/msgat_dataframe')

In [3]:
total_msg.loc[:10]

,msgSvrId,createTime,talker,sender,displayname,nickname,tfidf,intTime,createDate
0,4.996103e+18,2017-05-06 11:16:35,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,"{13: 0.23793951639049699, 242: 0.2672605677333...",29434.0,2017-05-06
1,7.180964e+18,2017-05-06 11:41:08,1002976072@chatroom,wangsiyu6227,NaN,NaN,"{199: 0.3430593633742228, 223: 0.2546946918464...",30907.0,2017-05-06
2,9.829728e+17,2017-05-06 11:43:42,1002976072@chatroom,wangsiyu6227,NaN,NaN,{1306: 1.0},31061.0,2017-05-06
3,1.268070e+18,2017-05-07 14:55:11,1002976072@chatroom,wxid_dh8oojwj4rtd22,NaN,NaN,"{15: 0.16704316005632647, 39: 0.17606891937720...",128950.0,2017-05-07
4,1.860004e+18,2017-05-07 14:55:32,1002976072@chatroom,wxid_dh8oojwj4rtd22,NaN,NaN,"{554: 0.25130336706210726, 3027: 0.61175299112...",128971.0,2017-05-07
5,8.202517e+18,2017-05-07 23:15:34,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,{3106: 1.0},158973.0,2017-05-07
6,1.453792e+18,2017-05-08 01:51:42,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,{2408: 1.0},168341.0,2017-05-08
7,1.869218e+18,2017-05-09 01:31:03,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,"{42: 0.07711831616842556, 74: 0.06024989709432...",253502.0,2017-05-09
8,7.890548e+18,2017-05-09 05:48:03,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,"{40: 0.09770981749459991, 72: 0.20169667278232...",268922.0,2017-05-09
9,4.042587e+18,2017-05-09 05:48:21,1002976072@chatroom,wxid_qnx97cl7i53l51,NaN,NaN,{607: 1.0},268940.0,2017-05-09


In [4]:
twindow = 3  # 这个应该用在全局的totalmsg表上，每次处理相应窗口的数据
divideN = 1500  # 对timediff进行缩放

threshold = 0.6

w_contextfree = 0.14
w_auther = 0.3
w_conver = 0.6
w_temp = 0.1

auther_scale = 2.347
conver_scale = 0.383
temporal_scale = 4.686

minGlobalDate = min(total_msg['createTime'])

# 使用Cython重写部分函数

In [5]:
%load_ext Cython

In [6]:
%%cython
cimport numpy as np
import numpy as np
import scipy.stats as st
cdef int divideN = 1500
#base functions
cdef dict sumDicts(dict1,dict2):
    """Sum dict1 with dict2, only 2.
    """
    return {k: dict1.get(k, 0) + dict2.get(k, 0) for k in dict1.keys() | dict2.keys()}

cdef dict sumTfidf(tfidfList):
    """Sum the new computed tfidf
    Args:
        tfidfList: A list of tfidf
    Return:
        the return list: expanded tfidf dict
    """
    rList = {}
    for tfidf in tfidfList:
        rList = sumDicts(tfidf, rList)
    return rList

cdef list probMultiTfidf(probArray, tfidfList):
    """Multiply the context tfidf with probability in the same thread.
    Args:
        probArray(Array): probability in the same thread of msg context
        tfidfList(List): the tfidfList of msg context
    Returns:
        The return tuple
    """
    return ([{k: v for k, v in zip(tfidfList[i].keys(), probArray[i] * np.array(list(tfidfList[i].values())))} for i in
             range(len(probArray))])
def tempProbC(row,t_scale, contextdf,w_temp):
    """Expand the context-free info of msgi with temporal context.

    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]

    Returns:
        The return vector. The expand vector repretation of msgi with info from its temporal context.

    """
    msgrowid = row[0]
    talker = row[3]
    msgdate = row[8]#intTime

    tempContext = contextdf[contextdf.index != msgrowid]#这里使用index
    timediff = tempContext['intTime'].values - msgdate
    timediff = timediff / divideN
    probArray = st.norm.pdf(timediff, scale=t_scale)
    tfidfList = tempContext['tfidf'].tolist()
    newTfidf = probMultiTfidf(probArray, tfidfList)
    result = {k:(v * w_temp) for k, v in sumTfidf(newTfidf).items()}
    filename = talker[:-9]
    #print(filename)
    #print(result)
    with open(r'../records/sample/tempExpand/temp_expand{}'.format(filename),'a') as f:
        f.write("{0},{1}\n".format(msgrowid,result))

In [7]:
def mExpandTempC(msgdate,talker):
    #maxdate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) + datetime.timedelta(hours=24 + twindow)
    #mindate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) - datetime.timedelta(hours=twindow)
    #maxdate = (maxdate - minGlobalDate).total_seconds()
    #mindate = (mindate - minGlobalDate).total_seconds()
    #还需要限制在同一个聊天室中
    #contextdf = total_msg[(total_msg.intTime > mindate) & (total_msg.intTime < maxdate) & (total_msg.talker == talker)]
    targetdf = total_msg[(total_msg.createDate == msgdate) & (total_msg.talker == talker)]
    contextdf = targetdf
    #[tempProb(row,temporal_scale,contextdf,w_temp) for row in targetdf.itertuples(index=False)]        
    #targetdf.apply(lambda row: tempProbC(row,temporal_scale,contextdf,w_temp),axis=1)
    #follow speeds up
    for row in targetdf.itertuples():
        tempProbC(row,temporal_scale,contextdf,w_temp)

In [8]:
def mExpandTempP(lt):
    msgdate,talker = lt
    #maxdate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) + datetime.timedelta(hours=24 + twindow)
    #mindate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) - datetime.timedelta(hours=twindow)
    #maxdate = (maxdate - minGlobalDate).total_seconds()
    #mindate = (mindate - minGlobalDate).total_seconds()
    #还需要限制在同一个聊天室中
    #contextdf = total_msg[(total_msg.intTime > mindate) & (total_msg.intTime < maxdate) & (total_msg.talker == talker)]
    targetdf = total_msg[(total_msg.createDate == msgdate) & (total_msg.talker == talker)]
    contextdf = targetdf
    #[tempProb(row,temporal_scale,contextdf,w_temp) for row in targetdf.itertuples(index=False)]        
    #targetdf.apply(lambda row: tempProbC(row,temporal_scale,contextdf,w_temp),axis=1)
    #follow speeds up
    for row in targetdf.itertuples():
        tempProbC(row,temporal_scale,contextdf,w_temp)

# 测试1000条total_msg耗时

注意在正式跑程序时将total_msg修改和删除相关的测试生成数据

1000条3.13s 10000条66s

In [9]:
#[mExpandTemp(msgdate,talker) for msgdate in total_msg['createTime'].dt.date.unique() for talker in total_msg['talker'].unique()]

In [10]:
def unittest():
    for msgdate in total_msg['createDate'].unique():
        for talker in total_msg['talker'].unique()[:2]:
            #mExpandTempC(msgdate,talker)
            targetdf = total_msg[(total_msg.createDate == msgdate) & (total_msg.talker == talker)]
            #use query
            #contextdf = total_msg.query('createDate == @msgdate and talker == @talker')
            contextdf = targetdf
            #[tempProb(row,temporal_scale,contextdf,w_temp) for row in targetdf.itertuples(index=False)]        
            #targetdf.apply(lambda row: tempProbC(row,temporal_scale,contextdf,w_temp),axis=1)
            #follow speeds up
            for row in targetdf.itertuples():
                print(row)
                tempProbC(row,temporal_scale,contextdf,w_temp)               

%load_ext line_profiler

%lprun -f unittest unittest()

In [ ]:
for msgdate in total_msg['createDate'].dt.date.unique():
    for talker in total_msg['talker'].unique():
        mExpandTempC(msgdate,talker)

# 并行计算

测试

In [31]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [32]:
dview = c[:]

build a load balanced view

In [33]:
lview = c.load_balanced_view()
lview.block = True

需要把需要的变量push到engines里

In [34]:
dview.push(dict(total_msg = total_msg,twindow = twindow,divideN = divideN,w_temp = w_temp,temporal_scale = temporal_scale))

<AsyncResult: _push>

In [35]:
%%px 
import pandas as pd
import time
import datetime
import scipy.stats as st
import numpy as np
import os

%%px
def test2(v):#要提交到engines中
    return v

def testp(i):
    v1,v2 = i
    return test2(v1)#这里调用的函数，因为没有在engine中定义，所以出现 了问题，不执行了

lview.map(testp,[(msgdate,talker) for msgdate in total_msg['createDate'].unique() for talker in total_msg['talker'].unique()])

In [36]:
%%px
%load_ext Cython

[stdout:0] 
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
[stdout:1] 
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
[stdout:2] 
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
[stdout:3] 
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [37]:
%%px
%%cython
cimport numpy as np
import numpy as np
import scipy.stats as st
cdef int divideN = 1500
#base functions
cdef dict sumDicts(dict1,dict2):
    """Sum dict1 with dict2, only 2.
    """
    return {k: dict1.get(k, 0) + dict2.get(k, 0) for k in dict1.keys() | dict2.keys()}

cdef dict sumTfidf(tfidfList):
    """Sum the new computed tfidf
    Args:
        tfidfList: A list of tfidf
    Return:
        the return list: expanded tfidf dict
    """
    rList = {}
    for tfidf in tfidfList:
        rList = sumDicts(tfidf, rList)
    return rList

cdef list probMultiTfidf(probArray, tfidfList):
    """Multiply the context tfidf with probability in the same thread.
    Args:
        probArray(Array): probability in the same thread of msg context
        tfidfList(List): the tfidfList of msg context
    Returns:
        The return tuple
    """
    return ([{k: v for k, v in zip(tfidfList[i].keys(), probArray[i] * np.array(list(tfidfList[i].values())))} for i in
             range(len(probArray))])
def tempProbC(row,t_scale, contextdf,w_temp):
    """Expand the context-free info of msgi with temporal context.

    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]

    Returns:
        The return vector. The expand vector repretation of msgi with info from its temporal context.

    """
    msgrowid = row[0]
    talker = row[3]
    msgdate = row[8]#intTime

    tempContext = contextdf[contextdf.index != msgrowid]#这里使用index
    timediff = tempContext['intTime'].values - msgdate
    timediff = timediff / divideN
    probArray = st.norm.pdf(timediff, scale=t_scale)
    tfidfList = tempContext['tfidf'].tolist()
    newTfidf = probMultiTfidf(probArray, tfidfList)
    result = {k:(v * w_temp) for k, v in sumTfidf(newTfidf).items()}
    filename = talker[:-9]
    #print(filename)
    #print(result)
    with open(r'../records/sample/tempExpand/temp_expand{}'.format(filename),'a') as f:
        f.write("{0},{1}\n".format(msgrowid,result))

In [ ]:
parallel_result = lview.map(mExpandTempP, [(msgdate,talker) for msgdate in total_msg['createDate'].unique() for talker in total_msg['talker'].unique()])

In [13]:
parallel_result

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310]

In [127]:
lbp_result = lview.map(mExpandTempC,[[msgdate,talker] for msgdate in total_msg['createDate'].unique() for talker in total_msg['talker'].unique()])

# 正式跑全部的数据，预计一个小时左右 实际情况是14h以上(而且这时候还差10w)

如果使用列表生成器，对于没有返回值的函数反而会拖慢

速度太慢，考虑使用并行处理；
每个群作为一个输出的文件，这样也能够看到进度；

为什么和预计的1小时不同，因为1w条和55w条，循环的执行次数不一样，即使看每一次执行，每个群每一天的说话次数也有较大差距；

In [ ]:
#[mExpandTemp(msgdate,talker) for msgdate in total_msg['createTime'].dt.date.unique() for talker in total_msg['talker'].unique()]

In [13]:
import os


def tail(f, lines=1, _buffer=4098):
    """Tail a file and get X lines from the end"""
    # place holder for the lines found
    lines_found = []

    # block counter will be multiplied by buffer
    # to get the block size from the end
    block_counter = -1

    # loop until we find X lines
    while len(lines_found) < lines:
        try:
            f.seek(block_counter * _buffer, os.SEEK_END)
        except IOError:  # either file is too small, or too many lines requested
            f.seek(0)
            lines_found = f.readlines()
            break

        lines_found = f.readlines()

        # we found enough lines, get out
        # Removed this line because it was redundant the while will catch
        # it, I left it for history
        # if len(lines_found) > lines:
        #    break

        # decrement the block counter to get the
        # next X bytes
        block_counter -= 1

    return lines_found[-lines:]

In [16]:
with open(r'../records/sample/temp_expand_test','r') as f:
    temptail = tail(f)

In [22]:
temptail[0].split(',',1)[1].count(':')

1729

# 下面加入auther_context和conver_context的处理函数

In [107]:
unittest()

Pandas(Index=0, msgSvrId=4.996103129789416e+18, createTime=Timestamp('2017-05-06 11:16:35'), talker='1002976072@chatroom', sender='wxid_qnx97cl7i53l51', displayname=nan, nickname=nan, tfidf={13: 0.23793951639049699, 242: 0.2672605677333875, 516: 0.22103512932743874, 539: 0.2941419485045756, 1034: 0.26238536866503787, 1040: 0.26307694010782456, 1250: 0.23905712440607488, 1251: 0.2739020265800637, 1252: 0.22991240099800486, 1253: 0.22071286669745097, 1254: 0.2793808000470636, 1255: 0.23837455929121726, 1256: 0.27379824532000974, 1257: 0.39362285433757105}, intTime=29434.0, createDate=Timestamp('2017-05-06 00:00:00'))
1002976072
{223: 0.0021212484260862224, 199: 0.0028572018102769187, 360: 0.0026045390680608568, 1289: 0.004023150674662093, 1290: 0.00295638386373971, 1291: 0.0034868607556696207, 1306: 0.00828845198463266, 1180: 0.0024882297082008216, 511: 0.0025774228445276967}
Pandas(Index=1, msgSvrId=7.180964081675663e+18, createTime=Timestamp('2017-05-06 11:41:08'), talker='1002976072@c

1002976072
{8192: 0.0004677514675665395, 8193: 0.0002848279627217715, 1: 0.00014070034157421247, 515: 0.0004247844363307654, 1028: 0.00015723809692007827, 1546: 0.001973713324671892, 5645: 0.0001778040283760959, 15: 0.0001363036168753467, 528: 0.003213825206613831, 2065: 0.0001891643971159283, 7700: 0.003334622900466404, 2072: 0.00018523157424211456, 2083: 0.00026006783071870016, 3111: 0.00016441052161641782, 41: 0.00170536650357365, 554: 0.00045469527269462575, 5162: 0.00067061452848179, 1076: 0.00013978731258341165, 2619: 0.004887109990486837, 3656: 0.0003949004501840698, 3657: 0.00016194861778568925, 74: 0.0002009092151002198, 1103: 0.0003097721612066678, 3157: 0.00016234920242116013, 4185: 4.5206508300719116e-05, 603: 0.00035786745874631133, 92: 0.002641521891695533, 606: 0.00015445283112066788, 97: 0.00626141625987783, 611: 0.00012396196169084148, 3172: 0.00017385945320574073, 100: 0.004139251616343406, 104: 0.0002573577988760228, 618: 0.00015227904757024143, 4714: 0.0002092762226

Pandas(Index=45, msgSvrId=5.383444357436017e+18, createTime=Timestamp('2017-05-12 02:45:26'), talker='1002976072@chatroom', sender='wxid_qnx97cl7i53l51', displayname=nan, nickname=nan, tfidf={13: 0.30661960085601586, 31: 0.2536792381112704, 39: 0.0756626452365537, 60: 0.11348853822279885, 351: 0.11147834523372403, 491: 0.15887752820463238, 497: 0.11110018761772254, 591: 0.18170046416091784, 614: 0.09051252784233511, 661: 0.1307430999388268, 869: 0.19260651895271472, 1018: 0.14175349776114007, 1028: 0.27604175538390574, 1200: 0.16070577622367768, 1252: 0.14813774881798497, 1253: 0.14221028124536927, 1381: 0.12794514457868858, 1566: 0.13889030348839526, 1667: 0.13584237594722787, 2099: 0.212369926478125, 2101: 0.2097980000131474, 2104: 0.15226886269969417, 2111: 0.2339131931546995, 2768: 0.1541083601708178, 4188: 0.14991536533995145, 6458: 0.23709640037251214, 7459: 0.17902268880523473, 10511: 0.18658627410645526, 11373: 0.1964575189446469, 11586: 0.29034220827885343}, intTime=517164.999

Pandas(Index=62, msgSvrId=3.669720936482655e+18, createTime=Timestamp('2017-05-15 02:06:55'), talker='1002976072@chatroom', sender='wxid_qnx97cl7i53l51', displayname=nan, nickname=nan, tfidf={}, intTime=774054.0000000013, createDate=Timestamp('2017-05-15 00:00:00'))
1002976072
{1027: 8.219234407380616e-05, 515: 0.00023998137494070636, 1667: 8.849436507413062e-05, 2565: 0.00011941091881870548, 4494: 9.185408566946091e-06, 1806: 0.0001077132078568327, 274: 5.924285009533278e-06, 3346: 0.00010042788890527494, 5525: 8.95235972749027e-06, 15128: 0.005023489737389485, 15129: 0.005857431450834034, 538: 0.00019549357538112668, 2073: 5.002175618201664e-05, 287: 0.00406620796490719, 16033: 0.00017692307008772452, 16034: 7.302614272583945e-05, 1315: 5.332111009483484e-06, 16036: 7.308316334211192e-05, 16037: 7.39118301523785e-05, 16038: 0.00013237053761236175, 420: 0.00011324975159127573, 294: 0.0004228972591866862, 39: 6.892562748231805e-05, 554: 4.530529491406177e-06, 810: 1.1206633570961783e-0

1002976072
{2817: 0.004852248819179147, 11650: 0.0035711920556090754, 1539: 0.0028605134562387008, 515: 0.004770685562510375, 4612: 0.0028945857298565544, 15: 0.012066023141362245, 3091: 0.004452599605362052, 2964: 0.0120334783907654, 12444: 0.016905636394727357, 284: 0.00848680475407435, 670: 0.009455345634392813, 31: 0.0014919596691216103, 1186: 0.006360959804573857, 2982: 0.003399027582877999, 6695: 0.004867903842921185, 1193: 0.007937932865989985, 554: 0.002427271887805059, 4653: 0.0036921927171301495, 3375: 0.004778191906214934, 47: 0.0018392608107277502, 4015: 0.0017586366063761065, 3384: 0.006427729575830388, 3769: 0.008165322978702072, 18746: 0.004284727954832109, 18747: 0.004506005603740349, 18748: 0.0043463901821906735, 60: 0.0020023750054986426, 61: 0.004561504242208835, 6205: 0.005275264465745052, 1085: 0.007942151433648051, 4414: 0.0037518334515995637, 1470: 0.0041386070023531635, 1987: 0.0031257931111106766, 1857: 0.014361328250938058, 1606: 0.0027418574738485254, 2374: 0

...

{44416: 2.3455659357283664e-10, 256: 3.76079897408506e-11, 260: 3.182959806277736e-11, 263: 6.184595397781296e-11, 5128: 1.0013118851743205e-10, 9: 2.535136792493994e-11, 10: 2.5333954232890433e-11, 266: 3.638938258460743e-11, 524: 7.757210154555873e-11, 12812: 1.0892951111096306e-10, 27276: 1.334108760207557e-10, 399: 7.77493838184402e-11, 267: 4.2354972159341917e-11, 529: 7.311559288393035e-11, 9874: 1.114406613448689e-10, 1427: 8.533142475795144e-11, 269: 3.780466410753458e-11, 39317: 1.288164560312116e-10, 273: 3.779801386110346e-11, 274: 3.404809833553113e-11, 659: 6.323730679962674e-11, 3347: 4.957388261580486e-11, 662: 4.4151304887637525e-11, 5403: 1.0390741829974344e-10, 5404: 8.92860550697509e-11, 6172: 4.863119375332247e-11, 36127: 1.2110857591012534e-10, 1315: 5.6418564929265966e-11, 1704: 1.4589091203536198e-10, 682: 4.3347699729822883e-11, 1451: 6.695517291391903e-11, 181: 2.8874503785751722e-11, 7737: 4.3930691403177435e-11, 44476: 5.6119418004561626e-11, 6981: 0.00490524

Pandas(Index=1407, msgSvrId=8.979166932859246e+18, createTime=Timestamp('2017-11-07 03:40:16'), talker='1021801216@chatroom', sender='wxid_ti5iiq0t8ari12', displayname=nan, nickname=nan, tfidf={15: 0.2489539116583997, 986: 0.6449975808409776, 1022: 0.4066153277757902, 3703: 0.5972135679947792}, intTime=15986055.000000788, createDate=Timestamp('2017-11-07 00:00:00'))
1021801216
{6656: 0.00011001154321576363, 4612: 0.00012044995802053223, 15: 0.002174846607481982, 27: 7.15576798793693e-05, 6685: 0.0005633166318671787, 8223: 3.9012008100288256e-22, 3106: 0.000769241152462518, 552: 0.00042788735453243385, 36399: 0.00013826474601616811, 6192: 8.249779091950981e-05, 560: 9.215234509189356e-05, 559: 9.687176876978492e-05, 51: 0.00015630295691022435, 7737: 9.407241026748573e-05, 2109: 3.6639291190721817e-06, 7745: 0.0002174035450444871, 3652: 3.046327700383719e-06, 69: 2.8097726536795686e-22, 2118: 6.513472781529885e-05, 71: 2.0138435021560355e-06, 1097: 0.0006119744796741659, 4690: 0.00012968

Pandas(Index=1437, msgSvrId=6.318626787657138e+18, createTime=Timestamp('2017-11-17 01:15:32'), talker='1021801216@chatroom', sender='wxid_ti5iiq0t8ari12', displayname=nan, nickname=nan, tfidf={70: 0.16631388312268108, 113: 0.07338391750368789, 229: 0.10606089522210896, 269: 0.12027769047469256, 274: 0.10832596266968865, 320: 0.11946778783070232, 399: 0.13436022319325808, 500: 0.10416759139924604, 605: 0.1355883517147806, 628: 0.1079144958972741, 871: 0.1117071156297456, 927: 0.11561290843810994, 1018: 0.1260029184710568, 1096: 0.1409136725227597, 1195: 0.11793532219953563, 1599: 0.35809847264925154, 2408: 0.12623999092766308, 3029: 0.15687199142114916, 3033: 0.21401847032828067, 3663: 0.14162624379977445, 4833: 0.16522698473097192, 4834: 0.3060370869409065, 4872: 0.14511608834929743, 5467: 0.20602365350231497, 5468: 0.20792252376051085, 5605: 0.17619898002015907, 8615: 0.18697397392762952, 17593: 0.3851655510496691, 38548: 0.221325596423844, 47649: 0.23480211759384753}, intTime=168413

1021801216
{2048: 0.0002059644347641395, 13: 6.637420780123797e-05, 4188: 0.00017994325954874213, 611: 4.710910773959701e-05, 13422: 0.00034734718967472717, 4401: 9.025995385612014e-05, 1667: 0.00011762366220767701, 4765: 8.546480473959539e-05, 3391: 0.00011557836668203822, 15048: 0.0005865091711548758, 6863: 7.972907955225943e-05, 3796: 6.199753779394619e-05, 4309: 8.485553483003952e-05, 2263: 0.0004361313650122088, 2264: 0.00043398415475659317, 731: 0.000400794823789592, 1255: 6.649556480966137e-05, 2280: 0.0008608328043216957, 2284: 0.0008729918973427696, 2293: 0.0013094878460141544, 2294: 0.0004364959486713848, 2842: 0.000505737978379209, 2843: 0.0005082227718228755, 2844: 0.002507033286381929, 2845: 0.0004967992321638782, 2847: 0.0005014066572763858, 2848: 0.0010028133145527716, 2849: 0.0010028133145527716, 2850: 0.000501216212644759, 2851: 0.0010028133145527716, 2852: 0.0010028133145527716, 2853: 0.0010028133145527716, 2854: 0.0010028133145527716, 2855: 0.0005101481059806973, 285

1021801216
{260: 2.2196403996368335e-08, 262: 1.4485737741310425e-06, 518: 4.017472113860351e-08, 648: 0.0013800884924681455, 9: 0.0009325069830639107, 10: 0.0009318664499974043, 263: 2.1564170828121297e-08, 15497: 2.207932458874754e-06, 397: 0.0014137616988027716, 44941: 0.002980587949723991, 5006: 2.7890281798775396e-08, 15: 1.9225081538094506e-07, 9743: 2.7536656855158614e-08, 5654: 0.0018722391249859065, 23318: 8.877248130187551e-06, 2968: 0.0035098377741521105, 150: 1.8866109768953906e-06, 668: 2.7057474960502574e-08, 669: 2.6127907202376792e-08, 286: 0.003202815276065078, 7838: 0.0021651236626520263, 670: 0.001258035134929331, 1055: 0.0016144895313875735, 8094: 2.814193489338206e-08, 295: 8.56060723215487e-06, 11179: 6.748368682311803e-07, 5421: 4.6644670791495186e-08, 3118: 2.2376303302704526e-06, 4399: 2.435330711835494e-08, 7739: 2.80077880275e-08, 1084: 0.0016417450358030652, 60: 2.2111112400747253e-08, 3773: 2.9459487432748604e-06, 191: 2.3205680824038332e-08, 17857: 0.00226

In [41]:
total_msg.values.nbytes

720000